# Install & import

In [2]:
!pip install numpy
!pip install pandas 
!pip install bvh

In [3]:
import pandas as pd
from bvh import Bvh
import os

# Load BVH file and convert to DataFrame

In [7]:
# File paths and names
data_path = "C:\\Users\\noamg\\OneDrive - post.bgu.ac.il\\Documents\\תואר שני\\שנה א סמסטר א\\XR\\Project\\bvh_data"  # Change file path
file = "TestTimecode"  # Change file name
bvh_file = f"{data_path}\\{file}.BVH"
csv_file = f"{data_path}\\{file}.csv"


csv_file_copy = f"C:\\GitHub\\AR_Balance_Project\\data\\{file}.csv"

In [8]:
# Check if CSV already exists
if os.path.exists(csv_file):
    print(f"CSV file already exists: {csv_file}")
    exit()

# Read BVH file
try:
    with open(bvh_file, "r") as f:
        mocopi_bvh = Bvh(f.read())
except FileNotFoundError:
    print(f"BVH file not found: {bvh_file}")
    exit()

# Extract joint names
joint_channels = []
for joint in mocopi_bvh.get_joints():
    joint = str(joint).split()[1]
    for channel in mocopi_bvh.joint_channels(joint):
        joint_channels.append(f"{joint}_{channel}")

# Verify joint and channel extraction
print(f"Extracted {len(joint_channels)} joint channels.")
print(f"Example channels: {joint_channels[:10]}")

# Extract motion data
motion_data = []
try:
    for frame in mocopi_bvh.frames:
        motion_data.append([float(value) for value in frame])
except Exception as e:
    print(f"Error extracting motion data: {e}")
    exit()

# Check if data matches expected columns
if len(joint_channels) != len(motion_data[0]):
    print(f"Mismatch between joint channels ({len(joint_channels)}) and motion data columns ({len(motion_data[0])}).")
    exit()

# Convert to DataFrame
df = pd.DataFrame(motion_data, columns=joint_channels)

# Add TimeCode column (the TimeCode is local, t0 is "0" and not 17:04 for example)
print(mocopi_bvh.frame_time)
frame_time = mocopi_bvh.frame_time  # Extract Frame Time from BVH
df['TimeCode'] = [i * frame_time for i in range(len(df))]

# Save to CSV to Computer
df.to_csv(csv_file, index=False)
print(f"Motion data saved to{csv_file} computer")

# Save to CSV to repository
df.to_csv(csv_file_copy, index=False)
print(f"Motion data saved to{csv_file} repository file")


Extracted 162 joint channels.
Example channels: ['root_Xposition', 'root_Yposition', 'root_Zposition', 'root_Zrotation', 'root_Xrotation', 'root_Yrotation', 'torso_1_Xposition', 'torso_1_Yposition', 'torso_1_Zposition', 'torso_1_Zrotation']
0.033333
Motion data saved toC:\Users\noamg\OneDrive - post.bgu.ac.il\Documents\תואר שני\שנה א סמסטר א\XR\Project\bvh_data\TestTimecode.csv computer
Motion data saved toC:\Users\noamg\OneDrive - post.bgu.ac.il\Documents\תואר שני\שנה א סמסטר א\XR\Project\bvh_data\TestTimecode.csv repository file
